In [2]:
import time
from functools import cache
from array import array

# Definiere Dekorierer Funktion um die Laufzeit von den Algorithmen ausgeben zu können.
def time_function(func):
    def wrapper(**kwargs):
        start = time.time()
        result = func(**kwargs)
        delta = time.time() - start
        print(f'Funktion {func.__name__} hat {delta} Sekunden benötigt!')
        return result
    return wrapper

In [3]:
@cache
def get_primes(n: int) -> list:
    # Für die Differenzen reichen shorts aus.
    p_dist = array("H")
    last_prime = 2
    # Sieb Implementation in der Vielfache eliminiert werden.
    # Es wird zur Speicherkostenbegrenzung nur der Abstand zur letzten Primzahl gespeichert.
    primes = [True] * (n-2)
    for number in range(2, n):
        if primes[number-2]:
            p_dist.append(int((number-last_prime)))
            last_prime = number
            for multiple in range(2 * number,n, number):
                primes[multiple - 2] = False
    return p_dist

# Lookup Table für nachfolgende Beispiele
primes_lut = get_primes(n=1000000000)

In [4]:
@time_function
def probe_division(number: int, interval: tuple):
    assert number > 1
    n = number
    prime_factor = 2
    factors = []
    for difference in primes_lut:
        # 1 ist trivialer Faktor
        if n == 1:
            break
        prime_factor = prime_factor + difference
        if prime_factor < interval[0]:
            continue
        # Produkt zweier Faktoren größer sqrt(n) wären größer als n.
        # Daher kann in dem Fall abgebrochen werden
        if prime_factor**2 > n or prime_factor >= interval[1]:
            factors.append(int(n))
            break
        # Faktor so lange rausdividieren, bis er n nicht mehr teilt.
        while n % prime_factor == 0:
            n = int(n/prime_factor)
            factors.append(prime_factor)
    return factors

# Beispiel aus dem Vortrag: 108598456355002
test_number = (sum(primes_lut)+2) * (sum(primes_lut[:-1])+2)
print(f'Zusammengesetzte Zahl: {test_number}')
probe_division(number=test_number, interval=(2,2000000000))

Zusammengesetzte Zahl: 999999866000004473
Funktion probe_division hat 12.323671579360962 Sekunden benötigt!


[999999929, 999999937]

In [5]:
@cache
def ggT(number1: int, number2: int):
    h = number1 % number2
    a,b = number2, h

    while b != 0:
        h = a % b
        a = b
        b = h
    return abs(a)

# Test mit 2 Primzahlen
assert ggT(99999989, 99999971) == 1
ggT(65536, 48)

16

In [7]:
from math import floor, ceil, sqrt

root_n = lambda x, n: x**(1./n) if 0 <= x else -(-x)**(1./n)

@time_function
def lehmann_factor(n: int):
    factors = probe_division(number=n, interval=(2, int(floor(root_n(n,3)))))
    if len(factors) > 1:
        print(f'Faktoren kleiner als {root_n(n,3)}')
        return factors
    for k in range(1, int(ceil(root_n(n,3))) + 1):
        for x in range( ceil(sqrt(4*k*n)), floor(sqrt(4*k*n) + root_n(n,6)/4*sqrt(k)) ):
            y = x**2 - 4*k*n
            if sqrt(y) % 1 == 0:
                return ggT(x+int(sqrt(y)),n)

prime_pair = 999999929, 999999937
composite = prime_pair[0]*prime_pair[1]
factor = lehmann_factor(n=composite)
assert factor in prime_pair
print(f'{factor} ist ein Faktor von {composite}')

lehmann_factor(n=17*23)

Funktion probe_division hat 0.018451929092407227 Sekunden benötigt!
Funktion lehmann_factor hat 0.01856064796447754 Sekunden benötigt!
999999937 ist ein Faktor von 999999866000004473
Funktion probe_division hat 4.291534423828125e-06 Sekunden benötigt!
Funktion lehmann_factor hat 3.170967102050781e-05 Sekunden benötigt!


17

In [ ]:
@time_function
def fermat_factor(n: int):
    return